# Beverage Quality Predictions with Logistic Regression

In this notebook we use a Logistic Regression model for predicting the quality of beverages with various specifications.

### Loading Necessary Libraries:

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import zscore
import warnings
warnings.filterwarnings("ignore")

### Downloading the dataset:

In [2]:
file_id = '1CwsYiq3UNMAs7iMhHeRjcs6L1eD1EIqU'
url = f'https://drive.google.com/uc?id={file_id}'
df = pd.read_csv(url)
df

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,1592,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1139,1593,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1140,1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1141,1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6


### Getting some sense of the dataset:

In [3]:
df.describe()

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000
mean,804.969379,8.311111,0.531339,0.268364,2.532152,0.086933,15.615486,45.914698,0.996730,3.311015,0.657708,10.442111,5.657043
std,463.997116,1.747595,0.179633,0.196686,1.355917,0.047267,10.250486,32.782130,0.001925,0.156664,0.170399,1.082196,0.805824
min,0.000000,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,411.000000,7.100000,0.392500,0.090000,1.900000,0.070000,7.000000,21.000000,0.995570,3.205000,0.550000,9.500000,5.000000
50%,794.000000,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,37.000000,0.996680,3.310000,0.620000,10.200000,6.000000
75%,1209.500000,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,61.000000,0.997845,3.400000,0.730000,11.100000,6.000000
max,1597.000000,15.900000,1.580000,1.000000,15.500000,0.611000,68.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


### First we preprocess the data, then split them, since we don't want to repeat the preprocessing steps:

### Dropping the Id field:

It's not informative.

In [4]:
df = df.drop(columns=["Id"])
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6


Checking for null values:

In [5]:
df.isnull().sum()

,0
fixed acidity,0
volatile acidity,0
citric acid,0
residual sugar,0
chlorides,0
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,0
sulphates,0


### Anomaly detection:

We use the Z-score method that finds the outlier given a threshold. It uses mean and standard deviation to calculate the Z-score. In here, we filter the samples that have Z-score larger than 3:

In [6]:
x = zscore(df['total sulfur dioxide'])
outliers = df[x.abs() > 3]
outliers

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
249,6.1,0.210,0.40,1.4,0.066,40.5,165.0,0.99120,3.25,0.59,11.9,6
366,8.5,0.655,0.49,6.1,0.122,34.0,151.0,1.00100,3.31,1.14,9.3,5
421,6.6,0.390,0.49,1.7,0.070,23.0,149.0,0.99220,3.12,0.50,11.5,6
452,9.6,0.880,0.28,2.4,0.086,30.0,147.0,0.99790,3.24,0.53,9.4,5
453,9.5,0.885,0.27,2.3,0.084,31.0,145.0,0.99780,3.24,0.53,9.4,5
460,6.7,0.420,0.27,8.6,0.068,24.0,148.0,0.99480,3.16,0.57,11.3,6
485,9.8,0.980,0.32,2.3,0.078,35.0,152.0,0.99800,3.25,0.48,9.4,5
760,7.9,0.300,0.68,8.3,0.050,37.5,278.0,0.99316,3.01,0.51,12.3,7
761,7.9,0.300,0.68,8.3,0.050,37.5,289.0,0.99316,3.01,0.51,12.3,7
1066,7.7,0.540,0.26,1.9,0.089,23.0,147.0,0.99636,3.26,0.59,9.7,5


removing the outliers and repeating for all features:

In [7]:
for column in df.columns:

    if column != "quality":

        x = zscore(df[column])
        outliers = pd.concat((outliers, df[x.abs() > 3]))


outliers = outliers.drop_duplicates()

df = df.drop(outliers.index)
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6


### Normalizing:

First, we check the new min, and max for all features:

In [8]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.00000,1055.000000,1055.000000,1055.000000
mean,8.290427,0.528924,0.260445,2.418815,0.081777,14.999052,43.539336,0.996708,3.31582,0.642863,10.428752,5.662559
std,1.663936,0.171309,0.191973,0.992481,0.023836,9.423466,29.305855,0.001770,0.14431,0.135747,1.035565,0.804324
min,5.000000,0.120000,0.000000,1.200000,0.012000,1.000000,6.000000,0.990640,2.88000,0.330000,8.400000,3.000000
25%,7.100000,0.397500,0.090000,1.900000,0.070000,7.000000,21.000000,0.995600,3.21000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.240000,2.200000,0.079000,13.000000,35.000000,0.996640,3.31000,0.620000,10.200000,6.000000
75%,9.100000,0.640000,0.420000,2.600000,0.089000,20.000000,58.000000,0.997800,3.40000,0.710000,11.100000,6.000000
max,15.000000,1.070000,0.790000,13.800000,0.415000,68.000000,144.000000,1.002420,3.78000,1.950000,14.000000,8.000000


Applying Min-Max normalization:

In [9]:
df['fixed acidity'] = (df['fixed acidity'] - 5) / (15 - 5)
df['citric acid'] = (df['citric acid']) / (0.79)
df['residual sugar'] = (df['residual sugar'] - 1.2) / (13.8 - 1.2)
df['chlorides'] = (df['chlorides'] - 0.012) / (0.415 - 0.012)
df['free sulfur dioxide'] = (df['free sulfur dioxide'] - 1) / (68 - 1)
df['total sulfur dioxide'] = (df['total sulfur dioxide'] - 6) / (144 - 6)
df['pH'] = (df['pH'] - 2.88) / (3.78 - 2.88)
df['sulphates'] = (df['sulphates'] - 0.33) / (1.95 - 0.33)
df['alcohol'] = (df['alcohol'] - 8.4) / (14 - 8.4)

df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000
mean,0.329043,0.528924,0.329678,0.096731,0.173145,0.208941,0.272024,0.996708,0.484244,0.193125,0.362277,5.662559
std,0.166394,0.171309,0.243004,0.078768,0.059147,0.140649,0.212361,0.001770,0.160345,0.083795,0.184922,0.804324
min,0.000000,0.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.990640,0.000000,0.000000,0.000000,3.000000
25%,0.210000,0.397500,0.113924,0.055556,0.143921,0.089552,0.108696,0.995600,0.366667,0.135802,0.196429,5.000000
50%,0.290000,0.520000,0.303797,0.079365,0.166253,0.179104,0.210145,0.996640,0.477778,0.179012,0.321429,6.000000
75%,0.410000,0.640000,0.531646,0.111111,0.191067,0.283582,0.376812,0.997800,0.577778,0.234568,0.482143,6.000000
max,1.000000,1.070000,1.000000,1.000000,1.000000,1.000000,1.000000,1.002420,1.000000,1.000000,1.000000,8.000000


### Separating features and labels

In [10]:
features = df.drop('quality', axis=1)
labels = df['quality']
print(f"features: \n{features.columns} \n\nlabels: \n{labels}")

features: 
Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object') 

labels: 
0       5
1       5
2       5
3       6
4       5
       ..
1138    6
1139    6
1140    5
1141    6
1142    5
Name: quality, Length: 1055, dtype: int64


### Dividing data into train and test sets:

In this case, we wish to use **Cross Entropy Loss (CE Loss)** which is common  for classification problems.

We separate data by 80%-20% for train and testing, meaning the size of the test set is 1/4 of the training set. We also define a random seed so we can have the same choice of samples for each class, no matter how many times we run this.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=200)
y_test

,quality
552,6
729,7
55,5
251,5
530,5
...,...
395,6
35,6
873,6
1049,6


### Implementing the model:

In [107]:
class LogisticRegressionModel:


    def __init__(self, input_dim, num_classes, alpha=0.01):

        self.w = np.random.randn(input_dim, num_classes) * 0.01
        self.b = np.zeros((1, num_classes))
        self.learning_rate = alpha



    def softmax(self, z):

        exp_values = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)



    def cross_entropy(self, y_true, y_prob):

        num_samples = y_true.shape[0]
        log_probs = -np.log(y_prob[range(num_samples), y_true - 3])
        return np.mean(log_probs)



    def compute_gradients(self, X, y_true, y_prob):

        num_samples = X.shape[0]
        y_true_encoded = np.zeros_like(y_prob)
        y_true_encoded[np.arange(num_samples), y_true - 3] = 1
        error = (y_prob - y_true_encoded) / num_samples
        grad_weights = np.dot(X.T, error)
        grad_biases = np.sum(error, axis=0, keepdims=True)
        return grad_weights, grad_biases



    def update_parameters(self, grad_weights, grad_biases):

        self.w -= self.learning_rate * grad_weights
        self.b -= self.learning_rate * grad_biases



    def fit(self, X, y, epochs):

        for epoch in range(epochs + 1):

            z = np.dot(X, self.w) + self.b
            y_prob = self.softmax(z)

            loss = self.cross_entropy(y, y_prob)

            grad_weights, grad_biases = self.compute_gradients(X, y, y_prob)
            self.update_parameters(grad_weights, grad_biases)

            if epoch % 100 == 0:
                print(f"Epoch {epoch} | Loss: {loss:.4f}")



    def predict(self, X):

        z = np.dot(X, self.w) + self.b
        return np.argmax(self.softmax(z), axis=1) + 3


In [108]:
model = LogisticRegressionModel(X_train.iloc[1].shape[0], y_train.drop_duplicates().shape[0], alpha=0.03)

model.fit(X_train, y_train, epochs=2000)

Epoch 0 | Loss: 1.7919
Epoch 100 | Loss: 1.2509
Epoch 200 | Loss: 1.1906
Epoch 300 | Loss: 1.1649
Epoch 400 | Loss: 1.1482
Epoch 500 | Loss: 1.1355
Epoch 600 | Loss: 1.1249
Epoch 700 | Loss: 1.1157
Epoch 800 | Loss: 1.1075
Epoch 900 | Loss: 1.1002
Epoch 1000 | Loss: 1.0935
Epoch 1100 | Loss: 1.0874
Epoch 1200 | Loss: 1.0818
Epoch 1300 | Loss: 1.0766
Epoch 1400 | Loss: 1.0718
Epoch 1500 | Loss: 1.0672
Epoch 1600 | Loss: 1.0630
Epoch 1700 | Loss: 1.0591
Epoch 1800 | Loss: 1.0553
Epoch 1900 | Loss: 1.0518
Epoch 2000 | Loss: 1.0485


In [109]:
y_pred = model.predict(X_test.to_numpy())
class_report = classification_report(y_test, y_pred)

print("\nClassification Report:\n", class_report)


Classification Report:
               precision    recall  f1-score   support

           4       0.00      0.00      0.00         5
           5       0.64      0.75      0.69        91
           6       0.47      0.58      0.52        85
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00         3

    accuracy                           0.55       211
   macro avg       0.22      0.26      0.24       211
weighted avg       0.46      0.55      0.51       211



Before diving in details of these results, we must check why we only have 5 classes in output:

In [79]:
y_train.drop_duplicates()

,quality
27,7
733,6
380,5
271,8
324,3
66,4


The reason that we got only 5 classes is Imbalancement in data. We can see that we only have 5 unique classes in our test data (instead of 10). This imbalancement also effects the result of our prediction. As shown in the classification report, we got 0 precision, recall and f1-score for classes [4, 7, 8] because the number of their samples was too little according to the other classes. So, in the training, common classes draw the model more towards optimizing themselves and we lose the rare classes. In fact, model learns that no matter what the input is, if it classifies it as common class, it will probably get a low loss. Hence our model fits more on those classes.

So as we said, model acts good on common classes like [5, 6] and has a good f1-score in them. Overall accuracy is 55% but doesn't represent the model's behaviour correctly, Since we have zero f1-scores for some classes and fair f1-scores for the other.

## Performance Improvement:

The results aren't quite satisfying and Softmax hasn't managed to assign good probablities to classes. One of the weaknesses of softmax is Data Imbalancement which we faced here. In Data Imbalancement, softmax often fails to classify the rare classes correctly since it treats all classes equally. We can modify the Softmax to have weights for the classes and so it can pay more attention to the rare classes.

So we redefine the model with modification in softmax:

In [111]:
class LogisticRegressionModel:

    def __init__(self, input_dim, num_classes, alpha=0.01, class_weights=None):

        self.w = np.random.randn(input_dim, num_classes) * 0.01
        self.b = np.zeros((1, num_classes))
        self.learning_rate = alpha
        self.class_weights = class_weights if class_weights is not None else np.ones(num_classes)


    def softmax(self, z):

        weighted_logits = z * self.class_weights
        exp_values = np.exp(weighted_logits - np.max(weighted_logits, axis=1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)


    def cross_entropy(self, y_true, y_prob):

        num_samples = y_true.shape[0]
        log_probs = -np.log(y_prob[range(num_samples), y_true - 3])
        return np.mean(log_probs)


    def compute_gradients(self, X, y_true, y_prob):

        num_samples = X.shape[0]
        y_true_encoded = np.zeros_like(y_prob)
        y_true_encoded[np.arange(num_samples), y_true - 3] = 1
        error = (y_prob - y_true_encoded) / num_samples
        grad_weights = np.dot(X.T, error)
        grad_biases = np.sum(error, axis=0, keepdims=True)
        return grad_weights, grad_biases


    def update_parameters(self, grad_weights, grad_biases):

        self.w -= self.learning_rate * grad_weights
        self.b -= self.learning_rate * grad_biases


    def fit(self, X, y, epochs):

        for epoch in range(epochs + 1):
            z = np.dot(X, self.w) + self.b
            y_prob = self.softmax(z)
            loss = self.cross_entropy(y, y_prob)
            grad_weights, grad_biases = self.compute_gradients(X, y, y_prob)
            self.update_parameters(grad_weights, grad_biases)
            if epoch % 100 == 0:
                print(f"Epoch {epoch} | Loss: {loss:.4f}")


    def predict(self, X):

        z = np.dot(X, self.w) + self.b
        return np.argmax(self.softmax(z), axis=1) + 3


In [114]:
model = LogisticRegressionModel(X_train.iloc[1].shape[0], y_train.drop_duplicates().shape[0], alpha=0.03, class_weights=np.array([12.0, 10.2, 0.7, 0.8, 10.0, 12.0]))

model.fit(X_train, y_train, epochs=2000)

Epoch 0 | Loss: 2.8877
Epoch 100 | Loss: 1.1501
Epoch 200 | Loss: 1.1152
Epoch 300 | Loss: 1.0914
Epoch 400 | Loss: 1.0734
Epoch 500 | Loss: 1.0590
Epoch 600 | Loss: 1.0471
Epoch 700 | Loss: 1.0371
Epoch 800 | Loss: 1.0284
Epoch 900 | Loss: 1.0209
Epoch 1000 | Loss: 1.0142
Epoch 1100 | Loss: 1.0082
Epoch 1200 | Loss: 1.0029
Epoch 1300 | Loss: 0.9980
Epoch 1400 | Loss: 0.9935
Epoch 1500 | Loss: 0.9895
Epoch 1600 | Loss: 0.9857
Epoch 1700 | Loss: 0.9822
Epoch 1800 | Loss: 0.9789
Epoch 1900 | Loss: 0.9759
Epoch 2000 | Loss: 0.9730


In [115]:
y_pred = model.predict(X_test.to_numpy())
class_report = classification_report(y_test, y_pred)

print("\nClassification Report:\n", class_report)


Classification Report:
               precision    recall  f1-score   support

           4       0.00      0.00      0.00         5
           5       0.65      0.74      0.69        91
           6       0.52      0.55      0.53        85
           7       0.71      0.44      0.55        27
           8       0.00      0.00      0.00         3

    accuracy                           0.60       211
   macro avg       0.37      0.35      0.35       211
weighted avg       0.58      0.60      0.58       211



As we can see, we had a 5% increase in accuracy and we could also get good precision and recall and f1-score for class [7]. This class had 0 precision and recall in the last model because of its few samples in the data. But in here, we assigned a weight (=10) for this class and weights (0.7, 0.8) for the common classes [5, 6]. This new method of weighting in softmax has helped model to better classify the uncommon classes and yet keep the good performance in the common classes.